Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Abstractive Summarization on CNN/DM Dataset using Transformers


### Summary

This notebook demonstrates how to fine tune Transformers models like [BART](https://arxiv.org/abs/1910.13461) and [T5](https://arxiv.org/abs/1910.10683) together with HuggingFace's [transformers library](https://github.com/huggingface/transformers)for abstractive text summarization. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.




### Before You Start

Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of steps. If QUICK_RUN = True, the notebook takes about 5 minutes to run on a VM with 1 Tesla K80 GPUs with 12GB GPU memory.

### Additional Notes

* **ROUGE Evalation**: To run rouge evaluation, please refer to the section of compute_rouge_perl in [summarization_evaluation.ipynb](./summarization_evaluation.ipynb) for setup.

* **Distributed Training**:
Please note that the jupyter notebook only allows to use pytorch [DataParallel](https://pytorch.org/docs/master/nn.html#dataparallel). Faster speed and larger batch size can be achieved with pytorch [DistributedDataParallel](https://pytorch.org/docs/master/notes/ddp.html)(DDP). Script [extractive_summarization_cnndm_distributed_train.py](./extractive_summarization_cnndm_distributed_train.py) shows an example of how to use DDP.



In [1]:
%load_ext autoreload

%autoreload 2
## Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = True

### Configuration


In [2]:
import os
import shutil
import sys
from tempfile import TemporaryDirectory
import time
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.dataset.cnndm import CNNDMSummarizationDataset
from utils_nlp.eval import compute_rouge_python, compute_rouge_perl
from utils_nlp.models.transformers.abstractive_summarization_bartt5 import (
    AbstractiveSummarizer)

from utils_nlp.models.transformers.datasets import SummarizationDataset
import nltk
from nltk import tokenize

import pandas as pd
import scrapbook as sb
import pprint
start_time = time.time()

/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm



### Configuration: choose the transformer model to be used

Several pretrained models have been made available by [Hugging Face](https://github.com/huggingface/transformers). For abstractive summarization, the following pretrained models are supported. 

In [3]:
pd.DataFrame({"model_name": AbstractiveSummarizer.list_supported_models()})

,model_name
0,bart-large
1,bart-large-mnli
2,bart-large-cnn
3,bart-large-xsum
4,t5-small
5,t5-base
6,t5-large
7,t5-3b
8,t5-11b


In [4]:
# Transformer model being used
# MODEL_NAME = "bart-large"
MODEL_NAME = "t5-small"
# notebook parameters
# the cache data path during find tuning
CACHE_DIR = "./t5_cache" #TemporaryDirectory().name
summarizer = AbstractiveSummarizer(MODEL_NAME, cache_dir=CACHE_DIR)

### Data Preprocessing

The dataset we used for this notebook is CNN/DM dataset which contains the documents and accompanying questions from the news articles of CNN and Daily mail. The highlights in each article are used as summary. The dataset consits of ~289K training examples, ~11K valiation examples and ~11K test examples.   The code in following cell will download the CNN/DM dataset listed at https://github.com/harvardnlp/sent-summary/.


In [5]:
# the data path used to save the downloaded data file
DATA_PATH = "./bartt5_cnndm" #TemporaryDirectory().name
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = 100
if not QUICK_RUN:
    TOP_N = -1

In [6]:
train_dataset, test_dataset = CNNDMSummarizationDataset(top_n=TOP_N, local_cache_path=DATA_PATH, raw=True)

100%|██████████| 489k/489k [00:07<00:00, 61.4kKB/s] 


In [7]:
test_dataset[0]['tgt_txt']

"<t> marseille prosecutor says `` so far no videos were used in the crash investigation '' despite media reports . </t> <t> journalists at bild and paris match are `` very confident '' the video clip is real , an editor says . </t> <t> andreas lubitz had informed his lufthansa training school of an episode of severe depression , airline says . </t>\n"

In [8]:
len(test_dataset)

100

Preprocess the data.

In [9]:
%%time
abs_sum_train = summarizer.processor.preprocess(train_dataset)
# torch.save(abs_sum_train,  os.path.join(DATA_PATH, "train_{0}_full.pt".format(MODEL_NAME)))
# abs_sum_train = torch.load(os.path.join(DATA_PATH, "train_{0}_full.pt".format(MODEL_NAME)))

CPU times: user 485 ms, sys: 7.85 ms, total: 493 ms
Wall time: 492 ms


In [10]:
abs_sum_test = summarizer.processor.preprocess(test_dataset)
# torch.save(abs_sum_test,  os.path.join(DATA_PATH, "test_{0}_full.pt".format(MODEL_NAME)))
# abs_sum_test = torch.load(os.path.join(DATA_PATH, "test_{0}_full.pt".format(MODEL_NAME)))

In [11]:
print(len(abs_sum_train))
print(len(abs_sum_test))

100
100


#### Inspect Data

In [12]:
abs_sum_train[0].keys()

dict_keys(['src', 'src_txt', 'tgt', 'tgt_txt', 'source_ids', 'source_mask', 'target_ids'])

In [13]:
abs_sum_train[0]

{'src': 'editor \'s note : in our behind the scenes series , cnn correspondents share their experiences in covering news and analyze the stories behind the events . here , soledad o\'brien takes users inside a jail where many of the inmates are mentally ill . an inmate housed on the " forgotten floor , " where many mentally ill inmates are housed in miami before trial . miami , florida ( cnn ) -- the ninth floor of the miami-dade pretrial detention facility is dubbed the " forgotten floor . " here , inmates with the most severe mental illnesses are incarcerated until they \'re ready to appear in court . most often , they face drug charges or charges of assaulting an officer -- charges that judge steven leifman says are usually " avoidable felonies . " he says the arrests often result from confrontations with police . mentally ill people often wo n\'t do what they \'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid ,

## Fine tune model
To start model fine-tuning, we need to specify the paramters as follows.

In [14]:
BATCH_SIZE_PER_GPU = 4
GRADIENT_ACCUMULATION_STEPS = 1
MAX_POS_LENGTH = 512

# GPU used for training
NUM_GPUS = torch.cuda.device_count()


# Learning rate
LEARNING_RATE=3e-5
MAX_GRAD_NORM=0.1

# How often the statistics reports show up in training, unit is step.
REPORT_EVERY=100
SAVE_EVERY=1000

# total number of steps for training
MAX_STEPS=100
# number of steps for warm up
WARMUP_STEPS=5e1
    
if not QUICK_RUN:
    MAX_STEPS=1000
    WARMUP_STEPS=5e2
    
# inference parameters
TEST_PER_GPU_BATCH_SIZE = 32
BEAM_SIZE = 3
 

In [15]:

summarizer.fit(
            abs_sum_train,
            num_gpus=NUM_GPUS,
            batch_size=BATCH_SIZE_PER_GPU*NUM_GPUS,
            gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
            max_steps=MAX_STEPS,
            learning_rate=LEARNING_RATE,
            max_grad_norm=MAX_GRAD_NORM,
            warmup_steps=WARMUP_STEPS,
            verbose=True,
            report_every=REPORT_EVERY,
        )


timestamp: 20/05/2020 19:11:57, average loss: 2.734279, time duration: 84.708848,
                            number of examples in current reporting: 400, step 100
                            out of total 100
saving through pytorch to ./t5_cache/fine_tuned/abssum_t5-small.pt


In [31]:
# save a finetuned model and load a previous saved model
"""
import torch
model_path = os.path.join(
        CACHE_DIR,
        "abssum_modelname_{0}_steps_{1}.pt".format(
            MODEL_NAME, MAX_STEPS
        ))
summarizer.save_model(global_step=MAX_STEPS, full_name=model_path)

summarizer = AbstractiveSummarizer(MODEL_NAME, cache_dir=CACHE_DIR)
summarizer.model.load_state_dict(torch.load(model_path, map_location="cpu")['model'])
"""

./t5_cache
saving through pytorch to ./t5_cache/abssum_modelname_t5-small_steps_100.pt


<All keys matched successfully>

### Model Evaluation

[ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)), or Recall-Oriented Understudy for Gisting Evaluation has been commonly used for evaluating text summarization.             
For the settings in this notebook with QUICK_RUN=True, you should get ROUGE scores close to the following numbers:

``
{'rouge-1': {'f': 0.31741527202292646,
             'p': 0.3455155118288276,
             'r': 0.3045104334747269},
 'rouge-2': {'f': 0.12227435906684982,
             'p': 0.13407308558314568,
             'r': 0.11687233771002672},
 'rouge-l': {'f': 0.23522707640246865,
             'p': 0.2558803081762467,
             'r': 0.22589352441506083}}
 ``           

In [17]:
source = []
target = []
for i in abs_sum_test:
    source.append(i["src_txt"]) 
    target.append(i['tgt'].replace("<t>","").replace("</t>", "").replace("\n", "")) 

In [18]:
%%time
prediction = summarizer.predict(abs_sum_test, 
                                num_gpus=NUM_GPUS, 
                                batch_size=TEST_PER_GPU_BATCH_SIZE*NUM_GPUS,
                                min_length=24, 
                                max_length=48,
                                num_beams=BEAM_SIZE)


dataset length is 100
CPU times: user 2min 16s, sys: 432 ms, total: 2min 16s
Wall time: 2min 16s


In [19]:
rouge_scores = compute_rouge_python(cand=prediction, ref=target)
pprint.pprint(rouge_scores)

Number of candidates: 100
Number of references: 100
{'rouge-1': {'f': 0.31741527202292646,
             'p': 0.3455155118288276,
             'r': 0.3045104334747269},
 'rouge-2': {'f': 0.12227435906684982,
             'p': 0.13407308558314568,
             'r': 0.11687233771002672},
 'rouge-l': {'f': 0.23522707640246865,
             'p': 0.2558803081762467,
             'r': 0.22589352441506083}}


In [20]:
prediction[0]

'french prosecutor brice robin says " so far no videos were used in the crash investigation. he adds that a person who has such a video needs to immediately give it to investigators.'

In [21]:
target[0]

' marseille prosecutor says " so far no videos were used in the crash investigation " despite media reports .   journalists at bild and paris match are " very confident " the video clip is real , an editor says .   andreas lubitz had informed his lufthansa training school of an episode of severe depression , airline says . '

In [22]:
# for testing
sb.glue("rouge_2_f_score", rouge_scores['rouge-2']['f'])

## Prediction on a single input sample

In [23]:
source = """
But under the new rule, set to be announced in the next 48 hours, Border Patrol agents would immediately return anyone to Mexico — without any detainment and without any due process — who attempts to cross the southwestern border between the legal ports of entry. The person would not be held for any length of time in an American facility.

Although they advised that details could change before the announcement, administration officials said the measure was needed to avert what they fear could be a systemwide outbreak of the coronavirus inside detention facilities along the border. Such an outbreak could spread quickly through the immigrant population and could infect large numbers of Border Patrol agents, leaving the southwestern border defenses weakened, the officials argued.
The Trump administration plans to immediately turn back all asylum seekers and other foreigners attempting to enter the United States from Mexico illegally, saying the nation cannot risk allowing the coronavirus to spread through detention facilities and Border Patrol agents, four administration officials said.
The administration officials said the ports of entry would remain open to American citizens, green-card holders and foreigners with proper documentation. Some foreigners would be blocked, including Europeans currently subject to earlier travel restrictions imposed by the administration. The points of entry will also be open to commercial traffic."""

In [24]:
small_test_dataset = SummarizationDataset(
    None,
    source=[source],
    n_processes=1
)
preprocessed_dataset =  summarizer.processor.preprocess(small_test_dataset)


In [25]:
preprocessed_dataset[0].keys()

dict_keys(['src', 'src_txt', 'source_ids', 'source_mask'])

In [26]:
prediction = summarizer.predict(preprocessed_dataset, num_gpus=0, batch_size=1, min_length=24, max_length=48,)

dataset length is 1


In [27]:
prediction[0]

'the person would not be held for any length of time in an American facility. border defenses are weakened, officials say. the new rule is set to be announced in the next 48 hours.'

In [28]:
print("Total notebook running time {}".format(time.time() - start_time))

Total notebook running time 264.78186321258545


## Clean up temporary folders

In [30]:
if os.path.exists(DATA_PATH):
    shutil.rmtree(DATA_PATH, ignore_errors=True)
if os.path.exists(CACHE_DIR):
    shutil.rmtree(CACHE_DIR, ignore_errors=True)
